In [ ]:
import tensorflow as tf
import numpy as np
import obspy
import tqdm
import os

Sequential = tf.keras.models.Sequential
Conv1D = tf.keras.layers.Conv1D
MaxPooling1D = tf.keras.layers.MaxPooling1D
Flatten = tf.keras.layers.Flatten
Dense = tf.keras.layers.Dense
Dropout = tf.keras.layers.Dropout
to_categorical = tf.keras.utils.to_categorical
tqdm = tqdm.tqdm

print(f"TensorFlow: {tf.__version__}")
print(f"Numpy: {np.__version__}")
print(f"Obspy: {obspy.__version__}")


def load_data(directory, max_length=100000, padding_method='zeros'):
    data = []
    labels = []

    for grade in ['Grade A', 'Grade B']:
        for filename in tqdm(os.listdir(os.path.join(directory, grade)), desc=f"Processing {grade} files"):
            st = obspy.read(os.path.join(directory, grade, filename))
            st = st.decimate(8)
            waveform = st[0].data

            # Pad or truncate to max_length
            if len(waveform) < max_length:
                waveform = np.pad(waveform, (0, max_length - len(waveform)))
                
            elif len(waveform) > max_length:
                waveform = waveform[:max_length]

            data.append(waveform)
            labels.append(1 if grade == 'Grade A' else 0)

    # Convert to NumPy arrays and reshape to 3D format for Conv1D
    data = np.array(data).reshape(-1, max_length, 1)
    labels = np.array(labels)

    return data, labels

X, y = load_data("./data")

model = tf.keras.Sequential([
    # Convolutional layer with 64 filters, kernel size of 3, ReLU activation, and input shape (20000, 1)
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(20000, 1)),

    # Batch normalization layer to stabilize training
    tf.keras.layers.BatchNormalization(),

    # Max pooling layer with pool size of 2 to reduce dimensionality
    tf.keras.layers.MaxPooling1D(pool_size=2),

    # GRU layer with 64 units, returning sequences for the next layer
    tf.keras.layers.GRU(64, return_sequences=True),

    # Another GRU layer with 32 units
    tf.keras.layers.GRU(32),

    # Dropout layer with a dropout rate of 0.2 to prevent overfitting
    tf.keras.layers.Dropout(0.2),

    # Flatten layer to convert the 3D output to a 1D vector
    tf.keras.layers.Flatten(),

    # Dense layer with 128 units and ReLU activation
    tf.keras.layers.Dense(128, activation='relu'),

    # Dropout layer with a dropout rate of 0.5 to prevent overfitting
    tf.keras.layers.Dropout(0.5),

    # Final dense layer with 1 unit and sigmoid activation for binary classification
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=1, batch_size=16, validation_split=0.2)

model.save("Model.h5")

In [ ]:
import tensorflow as tf
import numpy as np
import obspy
import tqdm
import os

Sequential = tf.keras.models.Sequential
Conv1D = tf.keras.layers.Conv1D
MaxPooling1D = tf.keras.layers.MaxPooling1D
Flatten = tf.keras.layers.Flatten
Dense = tf.keras.layers.Dense

Dropout = tf.keras.layers.Dropout
to_categorical = tf.keras.utils.to_categorical
tqdm = tqdm.tqdm

print(f"TensorFlow: {tf.__version__}")
print(f"Numpy: {np.__version__}")
print(f"Obspy: {obspy.__version__}")


def load_data(directory, max_length=100000, padding_method='zeros'):
    data = []
    labels = []

    for grade in ['Grade A', 'Grade B']:
        for filename in tqdm(os.listdir(os.path.join(directory, grade)), desc=f"Processing {grade} files"):
            
            st = obspy.read(os.path.join(directory, grade, filename))
            st = st.decimate(2)
            st = st.decimate(16)
            waveform = st[0].data

            # Pad or truncate to max_length
            if len(waveform) < max_length:
                waveform = np.pad(waveform, (0, max_length - len(waveform)))
                
            elif len(waveform) > max_length:
                waveform = waveform[:max_length]

            data.append(waveform)
            labels.append(1 if grade == 'Grade A' else 0)

    # Convert to NumPy arrays and reshape to 3D format for Conv1D
    data = np.array(data).reshape(-1, max_length, 1)
    labels = np.array(labels)

    return data, labels

X, y = load_data("./data")

# Create the model architecture
model = Sequential([

    Conv1D(32, kernel_size=5, activation='relu', input_shape=(20000, 1)),

    MaxPooling1D(pool_size=2),

    Conv1D(64, kernel_size=5, activation='relu'),

    MaxPooling1D(pool_size=2),

    Flatten(),

    Dense(128, activation='relu'),

    Dropout(0.5),

    Dense(1, activation='sigmoid')

])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2)

model.save("Model.h5")

In [ ]:
import tensorflow as tf
import numpy as np
import obspy
import tqdm
import os

model = tf.keras.models.load_model("./Model.h5")

def convert_mseed_file(mseed_file, max_length=100000, padding_method='zeros'):
    st = obspy.read(mseed_file)
    st = st.decimate(16)
    waveform = st[0].data

    # Pad or truncate to max_length
    if len(waveform) < max_length:
        if padding_method == 'zeros':
            waveform = np.pad(waveform, (0, max_length - len(waveform)))
        elif padding_method == 'repeat':
            waveform = np.tile(waveform, (max_length // len(waveform) + 1,))[:max_length]
    elif len(waveform) > max_length:
        waveform = waveform[:max_length]

    return waveform.reshape(1, max_length, 1)

def predict_mseed(mseed_file):
    converted_data = convert_mseed_file(mseed_file, max_length=100000, padding_method='zeros')
    prediction = model.predict(converted_data)
    print(prediction)
    predicted_grade = 'Grade A' if prediction[0][0] > 0.5 else 'Grade B'
    return predicted_grade

mseed_file = "./data/Grade B/xa.s12.00.mhz.1972-11-14HR00_evid00331.mseed"
predicted_grade = predict_mseed(mseed_file)
print(f"Predicted grade: {predicted_grade}")

mseed_file = "./data/Grade A/xa.s12.00.mhz.1970-01-19HR00_evid00002.mseed"
predicted_grade = predict_mseed(mseed_file)
print(f"Predicted grade: {predicted_grade}")